In this exercise you will use Python's hplib library to model heat pump performance across different climates and building insulation levels with the ultimate goal of comparing air and ground source heat pumps.

Note you do not need to be a programmer to do this exercise! code blocks are ready to run, and certain key variables that you can change are clearly marked in the code. simply change values after the '=' sign and press play!

This exercise will lay the groundwork for economic modelling that will happen later in the course. For now the goal is to understand how key energetic metrics can be found, which will later be used as inputs to derive price comparisons

In [2]:
# Import HPLib for heat pump performance calculations
import hplib

# Import pandas for data management and matplotlib for visualizations
import pandas as pd
import matplotlib.pyplot as plt

as a first step, lets see if we can get heating demand estimates on a monthly level. There are many ways to do this and a lot of resources online. You can also just look up numbers.

However, we will do this by just assuming there is a thermal energy loss per m2 per square meter is a linear function of the temperature difference between inside and outside and some insulation factor.

Monthly Heat demand (kWh/m2 * month) = (Tout - Tin) * Insulation Factor

In [3]:
# this function just returns the heat load (in kWh/m2) for a specific amount of time 
# If we enter 720 hours (one month) we will get the average value for that month

def heating_demand(insulation_level, indoor_temp, outdoor_temp, area, hours):
    # Base heat loss rate per m² at different insulation levels (in W/m²·K)
    heat_loss_rates = {'low': 2.0, 'medium': 1.0, 'high': 0.5}
    heat_loss_rate = heat_loss_rates[insulation_level]
    
    # Calculate thermal load based on temperature difference
    thermal_load = heat_loss_rate * area * (indoor_temp - outdoor_temp)
    
    # Convert to kWh and multiply by hours of operation
    return thermal_load * hours / 1000

In [5]:
# Define insulation levels (e.g., high, medium, low insulation) in kWh/m²/month
insulation_levels = {
    'high': 5,  # High insulation (e.g., passive house)
    'medium': 15,  # Medium insulation (e.g., modern building)
    'low': 30   # Low insulation (e.g., older building)
}

# Define average monthly temperatures for different climates (Celsius). This list is represents average temper
# over a year starting in January 
climates = {
    'mild': [10, 8, 5, 7, 12, 15, 18, 18, 14, 10, 7, 5],  # mild, year-round
    'cold': [-5, -3, 0, 3, 8, 12, 15, 13, 9, 5, 0, -2],   # colder winter, moderate summer
    'very_cold': [-10, -8, -5, -2, 5, 10, 12, 10, 6, 2, -5, -8]  # very cold winter, short summer
}


now we can play with some values here and compare to something from literature.
try changing the insulation levels and the average monthly temperatures in the code block above and hitting play (you have to hit play on the code block abova and below to refresh the numbers)

In [6]:
def calculate_cop(heat_pump_type, outdoor_temp):
    """
    Calculate COP for a geothermal or air-source heat pump based on outdoor temperature.
    `heat_pump_type`: 'geothermal' or 'air-source'
    `outdoor_temp`: Current outdoor temperature in Celsius
    """
    if heat_pump_type == 'geothermal':
        cop = hplib.get_parameters(model='Generic_top', group_id=1, t_in=-7, t_out=40, p_th=10000)[] # Replace with actual HPLib function for geothermal
    elif heat_pump_type == 'air-source':
        cop = hplib.air_source_COP(outdoor_temp)  # Replace with actual HPLib function for air-source
    return cop


In [7]:
# Initialize results dictionary
results = []

# Iterate over each insulation level and climate
for insulation, demand in insulation_levels.items():
    for climate, temps in climates.items():
        monthly_demand = demand  # Demand in kWh/m²/month based on insulation level
        for month, temp in enumerate(temps):
            # Calculate COPs for both types of heat pumps
            cop_geo = calculate_cop('geothermal', temp)
            cop_air = calculate_cop('air-source', temp)
            
            # Calculate energy consumption based on demand and COP
            energy_geo = monthly_demand / cop_geo
            energy_air = monthly_demand / cop_air

            # Append results
            results.append({
                'insulation': insulation,
                'climate': climate,
                'month': month + 1,
                'temp': temp,
                'cop_geo': cop_geo,
                'cop_air': cop_air,
                'energy_geo': energy_geo,
                'energy_air': energy_air
            })

# Convert results to DataFrame for analysis and visualization
df = pd.DataFrame(results)


AttributeError: module 'hplib' has no attribute 'geothermal_COP'

In [ ]:
# Plot energy consumption for each climate and insulation level
for insulation in insulation_levels.keys():
    for climate in climates.keys():
        # Filter data for the given insulation level and climate
        data = df[(df['insulation'] == insulation) & (df['climate'] == climate)]
        
        # Plot monthly energy consumption
        plt.figure(figsize=(10, 6))
        plt.plot(data['month'], data['energy_geo'], label='Geothermal', marker='o')
        plt.plot(data['month'], data['energy_air'], label='Air-Source', marker='o')
        
        # Add plot details
        plt.title(f'Energy Consumption ({insulation.capitalize()} Insulation, {climate.capitalize()} Climate)')
        plt.xlabel('Month')
        plt.ylabel('Energy Consumption (kWh/m²)')
        plt.legend()
        plt.grid(True)
        plt.show()
